# Azure OpenAI Service Function Calling 기초
Azure OpenAI Service에서는 Function Calling(함수 호출) 기능을 사용할 수 있다. 이 기능을 사용하면 사용자의 질문이나 지시를 특정 프로그램의 함수와 쉽게 연동할 수 있다. Function Calling을 지원하는 모델은 사전에 정의된 함수에 기반하여 지능적으로 호출해야 할 함수를 결정하고 데이터 출력을 구조화한다. ChatGPT 플러그인 등 외부 API를 호출해서 질문에 답변하는 어시스턴트를 생성할 수도 있다.

https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=python

## 사전 준비

이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:

- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 GPT-3.5 Turbo / GPT-4 모델
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI GPT-3.5 Turbo / GPT-4 모델의 배포 이름
  - OpenAI API 버전
- Python(이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.


## 패키지 설치


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 라이브러리 및 환경변수 불러오기


## Azure OpenAI 설정
Azure OpenAI와 연동을 위해 필요한 정보는 보안을 위해 하드코딩 하지 말고 환경변수나 [dotenv](https://pypi.org/project/python-dotenv/)로 불러오는 것을 권장한다.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

# 이 변수에는 모델을 배포했을 때 설정한 커스텀 이름을 입력한다.
#AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo"

## 함수 호출(지원 중단)
아래와 같이 함수를 호출하는 방법은 `2023-07-01-preview` API 버전에서 사용할 수 있고 `gpt-35-turbo`、`gpt-35-turbo-16k`、`gpt-4`、`gpt-4-32k` 모델의 `0613` 버전에서 동작한다.

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
messages= [
    {"role": "user", "content": "무신정권 기간 중에 최씨 정권의 시대를 연 인물을 알고 싶어."}
]

functions= [
    {
        "name": "PeopleSearchTool",
        "description": "한국사 인물 정보 검색에 특화된 도구입니다. 사용자 질문으로 검색 쿼리를 생성하여 검색합니다. 쿼리는 문자열만 받습니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "검색 쿼리"
                }
            },
            "required": [
                "query"
            ]
        }
    },
    {
        "name": "CafeSearchTool",
        "description": "무신과 연관된 카페를 검색하는 데 특화된 도구입니다. 카페 검색 쿼리에는 무신의 이름을 입력해주세요.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "검색 쿼리"
                }
            },
            "required": [
                "query"
            ]
        }
    },
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    functions = functions,
    function_call="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

모델이 함수 호출이 필요하다고 판단한 경우에는 API 응답에 `function_call`이라는 프로퍼티가 포함되어 있다. `function_call` 프로퍼티에는 호출할 함수의 이름과 해당 함수에 전달할 인수가 들어 있다. 인수는 JSON 문자열이다.


## 함수 호출(새 버전)
`2023-12-01-preview` 버전이 공개되면서 `functions`와 `function_call` 매개 변수는 지원이 중단됐다. 새 버전에서 `functions`는 `tools`로, `function_call`는 `tool_choice`로 대체할 수 있다.

In [ ]:
messages= [
    {"role": "user", "content": "무신정권 기간 중에 최씨 정권의 시대를 연 인물을 알고 싶어."}
]

tools= [
    {
        "type": "function",
        "function": {
            "name": "PeopleSearchTool",
            "description": "한국사 인물 정보 검색에 특화된 도구입니다. 사용자 질문으로 검색 쿼리를 생성하여 검색합니다. 쿼리는 문자열만 받습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색 쿼리"
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "CafeSearchTool",
            "description": "무신과 연관된 카페를 검색하는 데 특화된 도구입니다. 카페 검색 쿼리에는 무신의 이름을 입력해주세요.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색 쿼리"
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    }
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    tools = tools,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

In [ ]:
messages= [
    {"role": "user", "content": "무신정권 기간 중에 최씨 정권의 시대를 연 인물의 이름은 최충헌이었다. 이번에는 최충헌과 관련된 카페를 알고 싶어."}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    tools = tools,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))


## 병렬 함수 호출
병렬 함수 호출 기능을 사용하면 여러 함수를 동시에 호출하는 병렬 방식으로 실행 및 결과 획득이 가능하다. 이 기능을 사용하면 API 호출 횟수를 줄일 수 있어 전반적인 성능이 향상된다.

### 지원하는 모델
- `gpt-35-turbo (1106)`
- `gpt-35-turbo (0125)`
- `gpt-4 (1106-preview)`
- `gpt-4 (0125-preview)`

### 지원하는 API 버전
- `2023-12-01-preview`

지원하지 않는 모델은 한 번에 하나의 함수만 사용할 수 있다.

In [ ]:
from openai import AzureOpenAI
import os

client_new = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
messages= [
    {"role": "user", "content": "샌프란시스코, 서울, 파리의 날씨를 알려줘."}
]

tools= [
     {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "지정한 장소의 현재 날씨 확인",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시 혹은 주(예: San Francisco, CA)",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client_new.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    tools = tools,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

날씨 앱은 여러 장소의 날씨를 동시에 취득할 때가 많다. 가령 위 코드처럼 세 곳의 날씨를 호출하면 `tool_calls`라는 배열에 고유한 id를 가진 3개의 함수 호출이 포함된 채팅 완료 메시지가 반환된다. 이 같은 함수 호출에 응답하려면 `tools_calls`의 `id`를 참조하는 `tool_call_id`와 각 함수를 호출한 결과를 포함한 3개의 새 메시지를 대화에 추가하면 된다.

## 실행할 함수 정의하기
모델은 호출해야 할 함수만 반환한다. 사용자를 대신해서 세상에 영향을 미치는 행위(이메일 보내기, 웹 사이트에 글쓰기, 웹 쇼핑 등)를 하는 함수를 사용한다면 실행 전에 사용자 확인이 필요하도록 절차를 수립하는 것을 강력히 권장한다.

In [ ]:
import json
# 동일한 날씨를 반환하도록 하드코딩된 함수 예시 
# 프로덕션 환경에서는 백엔드 API나 외부 API를 사용한다.
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Seoul", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [ ]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls

## 함수 실행

In [ ]:
# Step 2: 모델이 함수를 호출하려는지 체크한다.
if tool_calls:
    # Step 3: 함수 호출
    # Note: JSON 응답이 항상 유효할 것이라 확신할 수는 없다.
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # 이 예시에서는 1개의 함수만 사용하지만 여러 함수를 사용할 수도 있다.
    messages.append(response_message)  # 어시스턴트 응답으로 messages를 확장
    # Step 4: 각각의 함수를 호출하고 응답 정보를 모델에 전송한다.
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit")
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # 함수 응답으로 messages를 확장
    #print(messages)
    second_response = client_new.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=messages,
    )  # 모델에서 새로운 응답을 취득하고, 그 안에서 함수의 응답을 확인한다.
    #print(second_response)
    print(second_response.choices[0].message.content)